In [1]:
import zipfile
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip 

In [2]:
zip_ref=zipfile.ZipFile('pizza_steak.zip','r')
zip_ref.extractall()
zip_ref.close()

In [3]:
# # After inspecting we found below
# 1. Dir structure:
# pizza_steak
#     train
#         steak
#             img1
#             img2
#             img3 
#             ..
#             ..
#             imgn
#         pizza
#             img1
#             img2
#             img3 
#             ..
#             ..
#             imgn
#     test
#         steak
#             img1
#             img2
#             img3 
#             ..
#             ..
#             imgn
#         pizza
#             img1
#             img2
#             img3 
#             ..
#             ..
#             imgn

# train_pizza_len=750
# train_steak_length=750
# test_pizza_length=250
# test_steak_length=250

In [4]:
# !ls /kaggle/working/pizza_steak/train/pizza

In [5]:
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import seaborn as sns
import glob
import os
import pandas as pd

In [6]:
data_dir=pathlib.Path('/kaggle/working/pizza_steak/train/')
class_names=np.array(sorted([item.name for item in data_dir.glob('*')]))
print(class_names)

In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
    # Setup target directory (we'll view images from here)
    target_folder = str(target_dir)+'/'+target_class

    # Get a random image path
    random_image = random.sample(os.listdir(target_folder), 1)

    # Read in the image and plot it using matplotlib
    img = mpimg.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off");

    print(f"Image shape: {img.shape}") # show the shape of the image
#     return img

In [8]:
view_random_image(data_dir,'steak')

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [10]:
#preprocess data
train_datagen=ImageDataGenerator(rescale=1./255)
valid_datagen=ImageDataGenerator(rescale=1./255)

test_dir="pizza_steak/test/"
train_dir="pizza_steak/train/"

# train_data = train_datagen.flow_from_directory(train_dir,
#                                                batch_size=32,
#                                                target_size=(224,244),
#                                                class_mode="binary",
#                                                seed=42)                                            
# valid_data = valid_datagen.flow_from_directory(test_dir,
#                                                batch_size=32,
#                                                target_size=(224,244),
#                                                class_mode="binary",
#                                                seed=42)
# # train_dir = "pizza_steak/train/"
# # test_dir = "pizza_steak/test/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

In [11]:
# build CNN model
tf.random.set_seed(42)
model_1=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=10,
                          kernel_size=3,
                          activation='relu',
                          input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2,
                             padding='valid'),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])


In [12]:
history_1=model_1.fit(train_data,
                      epochs=5,
                      steps_per_epoch=len(train_data),
                      validation_data=valid_data,
                      validation_steps=len(valid_data)
                     )

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "pizza_steak/train/"
test_dir = "pizza_steak/test/"

# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="binary", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

# Create a CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model_1 = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(filters=10, 
                         kernel_size=3, # can also be (3, 3)
                         activation="relu", 
                         input_shape=(224, 224, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

# Compile the model
model_1.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_1 = model_1.fit(train_data,
                        epochs=5,
                        steps_per_epoch=len(train_data),
                        validation_data=valid_data,
                        validation_steps=len(valid_data))

In [14]:
model_1.summary()

In [15]:
#model 2
tf.random.set_seed(42)
model_2=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224,3)),
    tf.keras.layers.Dense(4,activation='relu'),
    tf.keras.layers.Dense(4,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_2=model_2.fit(train_data,
                     epochs=5,
                     steps_per_epoch=len(train_data),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [16]:
model_2.summary()

In [17]:
model_1.summary()

In [18]:
#model 2
tf.random.set_seed(42)
model_3=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(224,224,3)),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_3=model_2.fit(train_data,
                     epochs=5,
                     steps_per_epoch=len(train_data),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [19]:
model_3.summary()

In [20]:
#preprocessing again
test_datagen=ImageDataGenerator(rescale=1./255)
train_datagen=ImageDataGenerator(rescale=1./255)

train_data=train_datagen.flow_from_directory(train_dir,
                                            seed=42,
                                            target_size=(224,224),
                                            batch_size=32,
                                            class_mode='binary')
test_data=test_datagen.flow_from_directory(test_dir,
                                          batch_size=32,
                                          target_size=(224,224),
                                          seed=42,
                                          class_mode='binary')



In [21]:
images,labels=train_data.next()
len(images),len(labels)

In [22]:
images[:2],images[0].shape

In [23]:
labels

In [24]:
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Activation
from tensorflow.keras import Sequential


In [25]:

model_4=Sequential([
    Conv2D(filters=10,
          kernel_size=3,
          strides=1,
          padding='valid',
          activation='relu',
          input_shape=(224,224,3)),
    Conv2D(10,3,activation='relu'),
    Conv2D(10,3,activation='relu'),
    Flatten(),
    Dense(1,activation='sigmoid')
    
])

In [26]:
model_4.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_4=model_4.fit(train_data,
                     epochs=5,
                     steps_per_epoch=len(train_data),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [27]:
pd.DataFrame(history_4.history).plot(figsize=(12,10))

In [28]:
def plot_loss_curves(history):
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    
    accuracy=history.history['accuracy']
    val_accuracy=history.history['val_accuracy']
    
    epochs=range(len(history.history['loss']))
    
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [29]:
plot_loss_curves(history_4)

In [30]:
#use more max poooling to avoid overfitting like in above

tf.random.set_seed(42)

model_5 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2), # reduce number of features by half
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(1, activation='sigmoid')
])


In [31]:

model_5.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_5=model_5.fit(train_data,
                     epochs=5,
                     steps_per_epoch=len(train_data),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [32]:
model_5.summary()

In [33]:
plot_loss_curves(history_5)

In [34]:
# still over fitting 
#try augumentation

In [35]:
train_datagen_augumented=ImageDataGenerator(rescale=1./255.,
                                         rotation_range=20,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2,
                                         horizontal_flip=True,
                                         shear_range=0.2,
                                         zoom_range=0.2
                                        )
train_datagen=ImageDataGenerator(rescale=1/255.)
test_datagen=ImageDataGenerator(rescale=1/255.)


In [36]:
train_data_augumented=train_datagen_augumented.flow_from_directory(train_dir,
                                                               batch_size=32,
                                                               target_size=(224,224),
                                                               class_mode='binary',
                                                               seed=42,
                                                               shuffle=False)
train_data=train_datagen.flow_from_directory(train_dir,
                                            batch_size=32,
                                            shuffle=False,
                                            seed=42,
                                            target_size=(224,224),
                                            class_mode='binary')
test_data=test_datagen.flow_from_directory(test_dir,
                                            batch_size=32,
                                            shuffle=False,
                                            seed=42,
                                            target_size=(224,224),
                                            class_mode='binary')

In [37]:
images,labels=train_data_augumented.next()
images_a,labels_a=train_data.next()
random_number=random.randint(0,32)
plt.imshow(images[random_number])
plt.figure()
plt.imshow(images_a[random_number])

In [38]:
#use more max poooling to avoid overfitting like in above

tf.random.set_seed(42)

model_6 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2), # reduce number of features by half
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(1, activation='sigmoid')
])

model_6.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_6=model_6.fit(train_data_augumented,
                     epochs=5,
                     steps_per_epoch=len(train_data_augumented),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [39]:
model_6.summary()

In [40]:
plot_loss_curves(history_6)

In [41]:
train_data_augumented_shuffled=train_datagen_augumented.flow_from_directory(train_dir,
                                                               batch_size=32,
                                                               target_size=(224,224),
                                                               class_mode='binary',
                                                               seed=42,
                                                               shuffle=True)

In [42]:
#use more max poooling to avoid overfitting like in above

tf.random.set_seed(42)

model_7 = Sequential([
  Conv2D(10, 3, activation='relu', input_shape=(224, 224, 3)),
  MaxPool2D(pool_size=2), # reduce number of features by half
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Conv2D(10, 3, activation='relu'),
  MaxPool2D(),
  Flatten(),
  Dense(1, activation='sigmoid')
])

model_7.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_7=model_7.fit(train_data_augumented_shuffled,
                     epochs=5,
                     steps_per_epoch=len(train_data_augumented_shuffled),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [43]:
model_7.summary()

In [44]:

plot_loss_curves(history_7)

In [45]:
# build CNN model
tf.random.set_seed(42)
model_8=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=10,
                          kernel_size=3,
                          activation='relu',
                          input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2,
                             padding='valid'),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model_8.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_8=model_8.fit(train_data_augumented_shuffled,
                     epochs=5,
                     steps_per_epoch=len(train_data_augumented_shuffled),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [46]:
model_8.summary()


In [47]:
plot_loss_curves(history_8)

In [48]:
# build CNN model
tf.random.set_seed(42)
model_9=tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=10,
                          kernel_size=3,
                          activation='relu',
                          input_shape=(224,224,3)),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2,
                             padding='valid'),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.Conv2D(10,3,activation='relu'),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model_9.compile(loss=tf.keras.losses.BinaryCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(),
               metrics=['accuracy'])
history_9=model_9.fit(train_data_augumented_shuffled,
                     epochs=20,
                     steps_per_epoch=len(train_data_augumented_shuffled),
                     validation_data=valid_data,
                     validation_steps=len(valid_data))

In [49]:
model_9.summary()

In [50]:
plot_loss_curves(history_9)

In [51]:
#predict the data


In [52]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-steak.jpeg 
steak=mpimg.imread('03-steak.jpeg')
plt.imshow(steak)

In [53]:
def prep_and_load_data(image_name,image_size=224):
    img=tf.io.read_file(image_name)
    img=tf.image.decode_image(img,channels=3)
    img=tf.image.resize(img,size=[image_size,image_size])
    img=img/255
    return img

In [54]:
steak=prep_and_load_data('03-steak.jpeg')
steak

In [59]:
steak=tf.expand_dims(steak,axis=0)

In [60]:
model_9.predict(steak)

In [61]:
class_names[int(tf.round(model_9.predict(steak))[0][0])]

In [64]:
def pred_and_plot(model,filename,class_names):
    img=prep_and_load_data(filename)
    preds=model.predict(tf.expand_dims(img,axis=0))
    preds_class=class_names[int(tf.round(preds)[0][0])]
    plt.imshow(img)
    plt.title('Prediction: {}'.format(preds_class))
    plt.axis(False)

In [65]:
pred_and_plot(model_9,"03-steak.jpeg",class_names)

In [66]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg 
pred_and_plot(model_8, "03-pizza-dad.jpeg", class_names)

Multiclass classificatioon 

In [67]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip 

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_all_data.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [68]:
train_dir = "10_food_classes_all_data/train/"
test_dir = "10_food_classes_all_data/test/"

In [69]:
import pathlib
import numpy as np
data_dir=pathlib.Path(train_dir)
class_names=np.array(sorted([item.name for item in data_dir.glob('*')]))
class_names


In [70]:
#view random image

view_random_image(train_dir,target_class=random.choice(class_names))

In [71]:
train_datagen=ImageDataGenerator(rescale=1/255.)
test_datagen=ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                               batch_size=32,
                                               class_mode='categorical',
                                               seed=42) # changed to categorical

test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=(224, 224),
                                              batch_size=32,
                                              seed=42,
                                              class_mode='categorical')


In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model_10=Sequential([
    Conv2D(10,3,activation='relu',input_shape=(224,224,3)),
    Conv2D(10,3,activation='relu'),
    MaxPool2D(),
    Conv2D(10,3,activation='relu'),
    Conv2D(10,3,activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(10,activation='softmax')
])
model_10.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_10=model_10.fit(train_data,
                       epochs=5,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=len(test_data)
                       )

In [73]:
model_10.evaluate(test_data)

In [74]:
model_10.summary()

In [75]:
plot_loss_curves(history_10)

In [76]:
#using a simplified model

model_11=Sequential([
    Conv2D(10,3,activation='relu',input_shape=(224,224,3)),
    MaxPool2D(),
    Conv2D(10,3,activation='relu'),
    MaxPool2D(),
    Flatten(),
    Dense(10,activation='softmax')
])
model_11.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_11=model_11.fit(train_data,
                       epochs=5,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=len(test_data)
                       )

In [77]:
plot_loss_curves(history_11)

In [78]:
model_11.summary()

In [79]:
model_11.evaluate(test_data)

In [80]:
train_datagen_augumented=ImageDataGenerator(rescale=1/255.,
                                           zoom_range=0.2,
                                           shear_range=0.2,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           horizontal_flip=True,
                                           rotation_range=20
                                           )
train_data_augumenetd=train_datagen_augumented.flow_from_directory(train_dir,
                                                                  batch_size=32,
                                                                  target_size=(224,224),
                                                                  shuffle=True,
                                                                  seed=42,
                                                                  class_mode='categorical')

In [81]:
model_12=tf.keras.models.clone_model(model_11)

In [82]:
model_12.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model
history_12 = model_12.fit(train_data_augumenetd, # use augmented data
                          epochs=5,
                          steps_per_epoch=len(train_data_augumenetd),
                          validation_data=test_data,
                          validation_steps=len(test_data))

In [83]:
model_12.summary()

In [84]:
plot_loss_curves(history_12)

In [85]:
model_12.evaluate(test_data)

In [86]:
model_13=tf.keras.models.clone_model(model_10)
model_13.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
history_13=model_13.fit(train_data_augumenetd,
                       epochs=10,
                       steps_per_epoch=len(train_data_augumenetd),
                       validation_data=test_data,
                       validation_steps=len(test_data)
                       )

In [87]:
model_13.summary()

In [88]:
plot_loss_curves(history_13)

In [89]:
model_13.evaluate(test_data)

In [90]:
# -q is for "quiet"
!wget -q https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-pizza-dad.jpeg
!wget -q https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-steak.jpeg
!wget -q https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-hamburger.jpeg
!wget -q https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-sushi.jpeg

In [91]:
pred_and_plot(model=model_12, 
              filename="03-steak.jpeg", 
              class_names=class_names)

In [92]:
pred_and_plot(model_12, "03-sushi.jpeg", class_names)

In [93]:
pred_and_plot(model_12, "03-pizza-dad.jpeg", class_names)

In [96]:
def pred_and_plot(model, filename, class_names):
    """
    Imports an image located at filename, makes a prediction on it with
    a trained model and plots the image with the predicted class as the title.
    """
    # Import the target image and preprocess it
    img = prep_and_load_data(filename)

    # Make a prediction
    pred = model.predict(tf.expand_dims(img, axis=0))

    # Get the predicted class
    if len(pred[0]) > 1: # check for multi-class
        pred_class = class_names[pred.argmax()] # if more than one output, take the max
    else:
        pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

    # Plot the image and predicted class
    plt.imshow(img)
    plt.title(f"Prediction: {pred_class}")
    plt.axis(False);

In [97]:
pred_and_plot(model_12, "03-steak.jpeg", class_names)

In [98]:
pred_and_plot(model_12, "03-sushi.jpeg", class_names)

In [99]:
pred_and_plot(model_12, "03-hamburger.jpeg", class_names)

In [100]:
model_12.save("saved_trained_model")

In [101]:
loaded_model_11 = tf.keras.models.load_model("saved_trained_model")
loaded_model_11.evaluate(test_data)

In [102]:
model_13.evaluate(test_data)

In [103]:
pred_and_plot(model_13, "03-steak.jpeg", class_names)

In [104]:
pred_and_plot(model_13, "03-sushi.jpeg", class_names)

In [105]:
pred_and_plot(model_13, "03-hamburger.jpeg", class_names)

In [106]:
model_13.save('better_model')

In [107]:
loaded_model_13 = tf.keras.models.load_model("better_model")
loaded_model_13.evaluate(test_data)